In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import unidecode
import spacy

c:\Users\bapti\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
top = pd.read_csv('top200.csv')

bottom = pd.read_csv('bottom200.csv')

In [10]:
top['lyrics'].head()

0    [Paroles de "TOUT VA BIEN" ft. Naps & Ninho] [...
1    [Paroles de "FADE UP" ft. Hamza & SCH] [Intro ...
2    [Paroles de "DIE"] [Intro] La mala est gangx E...
3    [Paroles de "PETETE"] [Intro] Pew Grr Pew pew ...
4    [Paroles de "Time Time"] [Couplet 1 : Squeezie...
Name: lyrics, dtype: object

In [11]:
bottom['lyrics'].head()

0    [Paroles de "BUMPY"] [Couplet 1] Mourir du P38...
1    [Paroles de "VALLON DES AUFFES"] [Intro] HuFel...
2    [Paroles de "PADDOCK"] [Intro] [?] MamÃ© [Coup...
3    [Paroles de "IMPACT"] [Intro] Gang gang gang g...
4    [Paroles de "GRA GRA BOOM" ft. Skread] [Intro]...
Name: lyrics, dtype: object

In [8]:
# disable chained assignments
pd.options.mode.chained_assignment = None 

In [9]:
def stop_words_french(text):
    stop_words = stopwords.words('french')
    text = [word for word in text if word not in stopwords.words('french') and len(word)>1]
    return text

def stop_words_english(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stopwords.words('english') and len(word)>1]
    return text

In [12]:
def add_lyrics_features(df):
    df['lyrics']=df['lyrics'].dropna()
    df['lyrics_nb_max'] = 0
    df['lyrics_nb_moy'] = 0
    df['lyrics_nb_std'] = 0
    i = 0
    while i < len(df['lyrics']):
        
        df['lyrics'][i] = str(df['lyrics'][i])

        if (df['lyrics'][i] != 'nan') and (len(df['lyrics'][i]) > 0):
            df['lyrics'][i] = unidecode.unidecode(df['lyrics'][i])
            df['lyrics'][i] = re.sub(r'\[.*?\]', '', df['lyrics'][i])
            carac = ["'", "-", "!", ".", "?", "(", ")"]
            for c in carac:
                df['lyrics'][i] = df['lyrics'][i].replace(c, ' ')
            df['lyrics'][i] = df['lyrics'][i].lower()
            df['lyrics'][i] = df['lyrics'][i].split(' ')
            #lyrics[i] = lemmatization(nlp_fr, lyrics[i])
            # lyrics[i] = lemmatization(nlp_en, lyrics[i])
            df['lyrics'][i] = stop_words_english(df['lyrics'][i])
            df['lyrics'][i] = stop_words_french(df['lyrics'][i])

            if (df['lyrics'][i] != 'nan') and (len(df['lyrics'][i]) > 0):
                d = {j:df['lyrics'][i].count(j) for j in df['lyrics'][i]}

                max = np.array(list(d.values())).max()
                mean = np.array(list(d.values())).mean()
                std = np.array(list(d.values())).std()

                df['lyrics_nb_max'][i] = max
                df['lyrics_nb_moy'][i] = mean
                df['lyrics_nb_std'][i] = std

        i +=1
    return df

In [13]:
top = add_lyrics_features(top)
top.head()

,title,artist,genre,rank,danceability,energy,key,loudness,mode,speechiness,...,time_signature,lyrics,winter,spring,summer,autumn,id,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std
0,TOUT VA BIEN (feat. Ninho & Naps),Alonzo,rap,1,0.660,0.720,3.0,-5.874,0.0,0.0753,...,4.0,"[eh, alonz, shk, shk, naps, mama, eh, eh, mama...",0,1,0,0,6OZwia8loN0aPS0vTvsBjR,20,1.482143,1.903860
1,FADE UP,ZEG P,autre,2,0.758,0.599,4.0,-5.990,0.0,0.0877,...,4.0,"[zeg, track, bitch, passa, toute, night, compt...",0,0,1,0,4ZpIuzx91EAPK3VimONbfB,22,2.233333,3.035164
2,DIE,Gazo,rap,3,0.695,0.630,8.0,-7.160,0.0,0.0350,...,4.0,"[mala, gangx, eh, bang, bang, eh, grr, grr, ba...",0,0,1,0,3D29kjUyWxsT3jUUTtARVQ,8,1.754902,1.331638
3,PETETE,Gambi,rap,4,0.752,0.669,10.0,-9.817,0.0,0.2510,...,4.0,"[pew, grr, pew, pew, pew, pew, skrt, rah, rah,...",0,0,1,0,0z3bi63SNZ5ylyHOzb81Uq,13,1.956204,2.071244
4,Time Time,Trei Degete,autre,5,0.715,0.799,0.0,-6.447,0.0,0.0433,...,4.0,"[sunt, fericit, ca, manac/nc, pepene, pepene, ...",0,0,0,1,5wKDPtbdggE1roeVp3UdXX,128,9.107143,23.391603


In [14]:
bottom = add_lyrics_features(bottom)
bottom.head()

,title,artist,genre,rank,danceability,energy,key,loudness,mode,speechiness,...,time_signature,lyrics,winter,spring,summer,autumn,id,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std
0,BUMPY,Alonzo,rap,1,0.713,0.820,5,-6.650,0,0.3670,...,4,"[mourir, p38, mourir, covid, covid, quand, bar...",0,1,0,0,1D8lLfcmm2kpW7CTdhrFN9,6,1.568966,0.873204
1,VALLON DES AUFFES,Alonzo,rap,2,0.644,0.798,5,-5.791,0,0.0961,...,4,"[hufel, beats, papa, miller, vais, tout, nique...",0,1,0,0,0mJHGKoDHfSkDxvzPbo6EV,31,1.971223,3.430591
2,PADDOCK,Alonzo,rap,3,0.694,0.689,0,-6.287,1,0.1050,...,3,"[mama, apra""s, violence, viendra, pleurs, biso...",0,1,0,0,2kqcLRUnITv4Ev30ygx2Yb,10,1.517241,1.254528
3,IMPACT,Gazo,rap,4,0.767,0.673,8,-5.843,0,0.2280,...,4,"[gang, gang, gang, gang, ethan, jackson, prod,...",0,0,1,0,1hlUHJgbdFdvZvG64a3W1g,12,1.925926,1.577447
4,GRA GRA BOOM (feat. Skread),Gazo,rap,5,0.668,0.773,5,-5.841,1,0.0648,...,4,"[mala, gangx, attends, die, venir, dire, tais,...",0,0,1,0,3aOzpOcVaAXCb2bkabANde,13,2.238532,1.990339


In [17]:
top.describe()

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,tempo,duration_ms,time_signature,winter,spring,summer,autumn,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std
count,1320.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,...,1319.000000,1319.000000,1319.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000,1320.000000
mean,660.500000,0.721116,0.646079,5.282790,-7.003566,0.391964,0.175331,0.291488,0.007608,0.153380,...,123.661030,196318.349507,3.957544,0.229545,0.209091,0.180303,0.381061,13.153030,1.773237,1.781843
std,381.195488,0.128060,0.147248,3.572734,2.246327,0.488374,0.137636,0.223102,0.058262,0.104487,...,25.372279,48481.258432,0.308728,0.420700,0.406814,0.384585,0.485831,12.355889,0.783129,1.540086
min,1.000000,0.234000,0.120000,0.000000,-21.189000,0.000000,0.025800,0.000307,0.000000,0.023200,...,45.130000,32107.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,330.750000,0.641500,0.549000,2.000000,-8.147500,0.000000,0.056800,0.110500,0.000000,0.092800,...,104.128500,167453.500000,4.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.425507,0.987234
50%,660.500000,0.742000,0.659000,6.000000,-6.670000,0.000000,0.129000,0.247000,0.000000,0.115000,...,125.135000,190373.000000,4.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.637837,1.421394
75%,990.250000,0.813500,0.754000,8.000000,-5.531500,1.000000,0.271500,0.420000,0.000043,0.173000,...,140.052500,215093.500000,4.000000,0.000000,0.000000,0.000000,1.000000,15.000000,1.962703,2.046225
max,1320.000000,0.973000,0.965000,11.000000,-2.018000,1.000000,0.828000,0.994000,0.956000,0.812000,...,201.898000,515387.000000,5.000000,1.000000,1.000000,1.000000,1.000000,195.000000,12.076923,23.391603


In [18]:
bottom.describe()

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,tempo,duration_ms,time_signature,winter,spring,summer,autumn,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std
count,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,...,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000
mean,526.772814,0.667792,0.592666,5.190114,-7.991121,0.471483,0.167567,0.331929,0.036128,0.165150,...,120.896301,199025.325095,3.912548,0.248099,0.176806,0.144487,0.430608,12.538973,1.682583,1.725147
std,304.174456,0.164372,0.177272,3.587957,3.055784,0.499424,0.153963,0.266656,0.157701,0.119171,...,27.803992,57487.667310,0.440520,0.432115,0.381686,0.351750,0.495397,15.534848,0.798526,1.565137
min,1.000000,0.063300,0.034600,0.000000,-25.521000,0.000000,0.024100,0.000030,0.000000,0.026100,...,61.466000,60123.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,263.750000,0.569750,0.500000,2.000000,-9.049500,0.000000,0.047700,0.109000,0.000000,0.097150,...,99.497000,167120.000000,4.000000,0.000000,0.000000,0.000000,0.000000,6.000000,1.369464,0.907678
50%,526.500000,0.704000,0.615000,5.000000,-7.305500,0.000000,0.101000,0.263500,0.000000,0.117000,...,121.974000,192045.500000,4.000000,0.000000,0.000000,0.000000,0.000000,9.000000,1.629477,1.353985
75%,790.250000,0.788000,0.713000,8.000000,-6.132000,1.000000,0.259250,0.505000,0.000101,0.193000,...,139.895250,220128.000000,4.000000,0.000000,0.000000,0.000000,1.000000,15.000000,1.971751,2.122267
max,1053.000000,0.973000,0.968000,11.000000,-2.021000,1.000000,0.956000,0.994000,0.965000,0.848000,...,202.008000,966360.000000,5.000000,1.000000,1.000000,1.000000,1.000000,282.000000,13.368421,23.244447


In [ ]:
def lemmatization(nlp, texte):
    i = 0
    for mot in texte:
        doc = nlp(mot)
        for token in doc:
            texte[i] = token.lemma_.lower()
            
        i += 1
            
    return texte

In [ ]:
nlp_fr = spacy.load('fr_core_news_md')
nlp_en = spacy.load('en_core_web_md')

In [ ]:
i = 0
while i < len(top['lyrics']):
    if len(top['lyrics'][i]) > 0:
        top['lyrics'][i] = lemmatization(nlp_fr, top['lyrics'][i])
        top['lyrics'][i] = lemmatization(nlp_en, top['lyrics'][i])
    i +=1